In [13]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.decomposition import PCA
from sklearn.feature_selection import SelectKBest, f_classif, mutual_info_classif
from sklearn.model_selection import train_test_split
import warnings
warnings.filterwarnings('ignore')

In [14]:
X = pd.read_csv("../merge/merged_x.csv")
y = pd.read_csv("../merge/merged_y.csv")

In [15]:
print("\n" + "🔧 "*35)
print("BOSQICH 2: ADVANCED FEATURE ENGINEERING")
print("🔧 "*35)



🔧 🔧 🔧 🔧 🔧 🔧 🔧 🔧 🔧 🔧 🔧 🔧 🔧 🔧 🔧 🔧 🔧 🔧 🔧 🔧 🔧 🔧 🔧 🔧 🔧 🔧 🔧 🔧 🔧 🔧 🔧 🔧 🔧 🔧 🔧 
BOSQICH 2: ADVANCED FEATURE ENGINEERING
🔧 🔧 🔧 🔧 🔧 🔧 🔧 🔧 🔧 🔧 🔧 🔧 🔧 🔧 🔧 🔧 🔧 🔧 🔧 🔧 🔧 🔧 🔧 🔧 🔧 🔧 🔧 🔧 🔧 🔧 🔧 🔧 🔧 🔧 🔧 


In [16]:
print("\n" + "="*70)
print("1️⃣  BAZAVIY FEATURE'LARNI TAHLIL QILISH")
print("="*70)

print(f"\nOriginal feature count: {X.shape[1]}")
print(f"Feature names: {list(X.columns) if hasattr(X, 'columns') else 'NumPy array'}")

# Agar DataFrame bo'lsa
if isinstance(X, pd.DataFrame):
    print("\nFeature types:")
    print(X.dtypes.value_counts())
    
    print("\nNumerical features:")
    numerical_features = X.select_dtypes(include=[np.number]).columns.tolist()
    print(numerical_features)
    
    print("\nCategorical features:")
    categorical_features = X.select_dtypes(exclude=[np.number]).columns.tolist()
    print(categorical_features)
else:
    print("\n⚠️  X is NumPy array. Converting to DataFrame for feature engineering...")
    X = pd.DataFrame(X, columns=[f'feature_{i}' for i in range(X.shape[1])])
    numerical_features = X.columns.tolist()
    categorical_features = []



1️⃣  BAZAVIY FEATURE'LARNI TAHLIL QILISH

Original feature count: 56
Feature names: ['loan_type', 'loan_amount', 'loan_term', 'interest_rate', 'loan_purpose', 'loan_to_value_ratio', 'origination_channel', 'loan_officer_id', 'marketing_campaign', 'age', 'annual_income', 'employment_length', 'employment_type', 'education', 'marital_status', 'num_dependents', 'state', 'regional_unemployment_rate', 'regional_median_income', 'regional_median_rent', 'housing_price_index', 'cost_of_living_index', 'previous_zip_code', 'num_credit_accounts', 'oldest_credit_line_age', 'oldest_account_age_months', 'total_credit_limit', 'num_delinquencies_2yrs', 'num_inquiries_6mo', 'recent_inquiry_count', 'num_public_records', 'num_collections', 'account_diversity_index', 'application_hour', 'application_day_of_week', 'preferred_contact', 'account_status_code', 'num_login_sessions', 'num_customer_service_calls', 'has_mobile_app', 'paperless_billing', 'account_tenure', 'is_referred', 'monthly_income', 'existing_m

In [17]:
print("\n" + "="*70)
print("2️⃣  FEATURE ENGINEERING FUNKSIYALARI")
print("="*70)

def create_advanced_features(df, feature_config=None):
    """
    Kengaytirilgan feature'lar yaratish
    
    Parameters:
    -----------
    df : DataFrame
        Original data
    feature_config : dict
        Feature yaratish konfiguratsiyasi
    """
    df_enhanced = df.copy()
    new_features = []
    
    print("\n🔨 Feature Engineering boshlandi...")
    
    # 1. POLYNOMIAL FEATURES (muhim numerical features uchun)
    print("\n1. Polynomial features yaratilmoqda...")
    numerical_cols = df_enhanced.select_dtypes(include=[np.number]).columns.tolist()
    
    if len(numerical_cols) >= 2:
        # Faqat eng muhim 5-10 ta feature uchun
        important_numerical = numerical_cols[:min(5, len(numerical_cols))]
        
        for col in important_numerical:
            # Squared
            df_enhanced[f'{col}_squared'] = df_enhanced[col] ** 2
            new_features.append(f'{col}_squared')
            
            # Cube (ba'zi hollarda)
            if df_enhanced[col].std() > 0.1:  # Agar variability yuqori bo'lsa
                df_enhanced[f'{col}_cubed'] = df_enhanced[col] ** 3
                new_features.append(f'{col}_cubed')
            
            # Square root (faqat positive values uchun)
            if (df_enhanced[col] >= 0).all():
                df_enhanced[f'{col}_sqrt'] = np.sqrt(df_enhanced[col])
                new_features.append(f'{col}_sqrt')
            
            # Log transform (faqat positive values uchun)
            if (df_enhanced[col] > 0).all():
                df_enhanced[f'{col}_log'] = np.log1p(df_enhanced[col])
                new_features.append(f'{col}_log')
    
    print(f"   ✓ {len([f for f in new_features if any(x in f for x in ['squared', 'cubed', 'sqrt', 'log'])])} polynomial features")
    # 2. INTERACTION FEATURES (2 ta feature o'rtasida)
    print("\n2. Interaction features yaratilmoqda...")
    if len(numerical_cols) >= 2:
        # Eng muhim kombinatsiyalar
        for i in range(min(3, len(numerical_cols))):
            for j in range(i+1, min(4, len(numerical_cols))):
                col1, col2 = numerical_cols[i], numerical_cols[j]
                
                # Multiplication
                df_enhanced[f'{col1}_x_{col2}'] = df_enhanced[col1] * df_enhanced[col2]
                new_features.append(f'{col1}_x_{col2}')
                
                # Division (avoid division by zero)
                if (df_enhanced[col2] != 0).all():
                    df_enhanced[f'{col1}_div_{col2}'] = df_enhanced[col1] / (df_enhanced[col2] + 1e-5)
                    new_features.append(f'{col1}_div_{col2}')
    
    print(f"   ✓ {len([f for f in new_features if '_x_' in f or '_div_' in f])} interaction features")
    
    # 3. RATIO FEATURES (nisbatlar)
    print("\n3. Ratio features yaratilmoqda...")
    if len(numerical_cols) >= 2:
        for i, col1 in enumerate(numerical_cols[:3]):
            for col2 in numerical_cols[i+1:4]:
                if (df_enhanced[col1] + df_enhanced[col2]).std() > 0:
                    df_enhanced[f'{col1}_to_{col2}_ratio'] = (
                        df_enhanced[col1] / (df_enhanced[col1] + df_enhanced[col2] + 1e-5)
                    )
                    new_features.append(f'{col1}_to_{col2}_ratio')
    
    print(f"   ✓ {len([f for f in new_features if '_ratio' in f])} ratio features")
    
    # 4. BINNING (kategoriyalash)
    print("\n4. Binning features yaratilmoqda...")
    for col in numerical_cols[:5]:
        try:
            df_enhanced[f'{col}_bin'] = pd.qcut(
                df_enhanced[col], q=5, labels=False, duplicates='drop'
            )
            new_features.append(f'{col}_bin')
        except:
            pass
    
    print(f"   ✓ {len([f for f in new_features if '_bin' in f])} binning features")
    
    # 5. STATISTICAL FEATURES (agar multiple related columns bo'lsa)
    print("\n5. Statistical aggregation features yaratilmoqda...")
    if len(numerical_cols) >= 3:
        # Row-wise statistics
        df_enhanced['row_mean'] = df_enhanced[numerical_cols[:10]].mean(axis=1)
        df_enhanced['row_std'] = df_enhanced[numerical_cols[:10]].std(axis=1)
        df_enhanced['row_max'] = df_enhanced[numerical_cols[:10]].max(axis=1)
        df_enhanced['row_min'] = df_enhanced[numerical_cols[:10]].min(axis=1)
        df_enhanced['row_median'] = df_enhanced[numerical_cols[:10]].median(axis=1)
        
        new_features.extend(['row_mean', 'row_std', 'row_max', 'row_min', 'row_median'])
    
    print(f"   ✓ 5 statistical aggregation features")
    
    # 6. MISSING VALUE INDICATORS (agar missing values bo'lsa)
    print("\n6. Missing value indicators yaratilmoqda...")
    missing_cols = df_enhanced.columns[df_enhanced.isnull().any()].tolist()
    for col in missing_cols:
        df_enhanced[f'{col}_is_missing'] = df_enhanced[col].isnull().astype(int)
        new_features.append(f'{col}_is_missing')
    
    print(f"   ✓ {len(missing_cols)} missing indicators")
    
    # Fill missing values
    df_enhanced = df_enhanced.fillna(df_enhanced.median(numeric_only=True))
    
    print(f"\n{'='*70}")
    print(f"✅ FEATURE ENGINEERING YAKUNLANDI!")
    print(f"   Original features: {df.shape[1]}")
    print(f"   New features created: {len(new_features)}")
    print(f"   Total features: {df_enhanced.shape[1]}")
    print(f"{'='*70}")
    
    return df_enhanced, new_features




2️⃣  FEATURE ENGINEERING FUNKSIYALARI


In [18]:
print("\n" + "="*70)
print("3️⃣  FEATURE ENGINEERING QO'LLASH")
print("="*70)

# Original train/test split'dan oldin feature engineering
X_enhanced, new_features_list = create_advanced_features(X)

print(f"\n📊 Enhanced dataset shape: {X_enhanced.shape}")

# Yangi train/test split
X_train_enh, X_test_enh, y_train_enh, y_test_enh = train_test_split(
    X_enhanced, y, test_size=0.2, stratify=y, random_state=42
)

X_train_split_enh, X_val_enh, y_train_split_enh, y_val_enh = train_test_split(
    X_train_enh, y_train_enh, test_size=0.2, stratify=y_train_enh, random_state=42
)

print(f"✓ Enhanced Train: {X_train_split_enh.shape}")
print(f"✓ Enhanced Val: {X_val_enh.shape}")
print(f"✓ Enhanced Test: {X_test_enh.shape}")



3️⃣  FEATURE ENGINEERING QO'LLASH

🔨 Feature Engineering boshlandi...

1. Polynomial features yaratilmoqda...
   ✓ 17 polynomial features

2. Interaction features yaratilmoqda...
   ✓ 10 interaction features

3. Ratio features yaratilmoqda...
   ✓ 6 ratio features

4. Binning features yaratilmoqda...
   ✓ 5 binning features

5. Statistical aggregation features yaratilmoqda...
   ✓ 5 statistical aggregation features

6. Missing value indicators yaratilmoqda...
   ✓ 0 missing indicators

✅ FEATURE ENGINEERING YAKUNLANDI!
   Original features: 56
   New features created: 43
   Total features: 99

📊 Enhanced dataset shape: (89999, 99)
✓ Enhanced Train: (57599, 99)
✓ Enhanced Val: (14400, 99)
✓ Enhanced Test: (18000, 99)


In [19]:
print("\n" + "="*70)
print("4️⃣  FEATURE SELECTION (TOP FEATURES)")
print("="*70)

# Method 1: Mutual Information
print("\n📊 Method 1: Mutual Information...")
mi_selector = SelectKBest(score_func=mutual_info_classif, k='all')
mi_selector.fit(X_train_split_enh, y_train_split_enh)
mi_scores = pd.DataFrame({
    'feature': X_train_split_enh.columns,
    'mi_score': mi_selector.scores_
}).sort_values('mi_score', ascending=False)

print("\nTop 20 features (Mutual Information):")
print(mi_scores.head(20).to_string(index=False))

# Method 2: F-statistic (ANOVA)
print("\n📊 Method 2: F-statistic (ANOVA)...")
f_selector = SelectKBest(score_func=f_classif, k='all')
f_selector.fit(X_train_split_enh, y_train_split_enh)
f_scores = pd.DataFrame({
    'feature': X_train_split_enh.columns,
    'f_score': f_selector.scores_
}).sort_values('f_score', ascending=False)

print("\nTop 20 features (F-statistic):")
print(f_scores.head(20).to_string(index=False))

# Top K features tanlash (masalan top 100 yoki 150)
k_best = min(100, X_train_split_enh.shape[1])  # Maksimum 100 ta feature
print(f"\n🎯 Selecting top {k_best} features...")

# Mutual Information bo'yicha tanlash
top_features = mi_scores.head(k_best)['feature'].tolist()

X_train_split_selected = X_train_split_enh[top_features]
X_val_selected = X_val_enh[top_features]
X_test_selected = X_test_enh[top_features]

print(f"✓ Selected features shape: {X_train_split_selected.shape}")



4️⃣  FEATURE SELECTION (TOP FEATURES)

📊 Method 1: Mutual Information...

Top 20 features (Mutual Information):
                         feature  mi_score
          monthly_free_cash_flow  0.020032
                   annual_income  0.017329
                  monthly_income  0.015925
           loan_to_annual_income  0.015868
                             age  0.015691
         payment_to_income_ratio  0.014758
              debt_service_ratio  0.014398
       loan_type_x_interest_rate  0.011922
               preferred_contact  0.011611
          oldest_credit_line_age  0.010976
               paperless_billing  0.010810
loan_term_to_interest_rate_ratio  0.010539
                  has_mobile_app  0.010436
loan_type_to_interest_rate_ratio  0.010416
                available_credit  0.010254
     loan_type_div_interest_rate  0.009919
              total_credit_limit  0.009740
       oldest_account_age_months  0.009611
              num_login_sessions  0.009417
       loan_term_x_interest

In [20]:
print("\n" + "="*70)
print("5️⃣  FEATURE SCALING")
print("="*70)

scaler = StandardScaler()
X_train_split_scaled = pd.DataFrame(
    scaler.fit_transform(X_train_split_selected),
    columns=X_train_split_selected.columns,
    index=X_train_split_selected.index
)

X_val_scaled = pd.DataFrame(
    scaler.transform(X_val_selected),
    columns=X_val_selected.columns,
    index=X_val_selected.index
)

X_test_scaled = pd.DataFrame(
    scaler.transform(X_test_selected),
    columns=X_test_selected.columns,
    index=X_test_selected.index
)

print("✓ Scaling completed!")
print(f"  Mean: {X_train_split_scaled.mean().mean():.6f}")
print(f"  Std:  {X_train_split_scaled.std().mean():.6f}")



5️⃣  FEATURE SCALING
✓ Scaling completed!
  Mean: -0.000000
  Std:  1.000009


In [22]:
print("\n" + "="*70)
print("6️⃣  YANGI FEATURE'LAR BILAN ENSEMBLE MODEL")
print("="*70)

from catboost import CatBoostClassifier
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from sklearn.metrics import roc_auc_score, average_precision_score

print("\n🚀 CatBoost training...")
catboost_v2 = CatBoostClassifier(
    iterations=500,
    learning_rate=0.05,
    depth=8,
    l2_leaf_reg=3,
    class_weights=[1, 20],
    random_seed=42,
    verbose=False,
    early_stopping_rounds=50
)
catboost_v2.fit(
    X_train_split_scaled, y_train_split_enh,
    eval_set=(X_val_scaled, y_val_enh),
    verbose=False
)

y_proba_cb_v2 = catboost_v2.predict_proba(X_val_scaled)[:, 1]
roc_auc_cb_v2 = roc_auc_score(y_val_enh, y_proba_cb_v2)
pr_auc_cb_v2 = average_precision_score(y_val_enh, y_proba_cb_v2)
print(f"   ROC-AUC: {roc_auc_cb_v2:.4f} | PR-AUC: {pr_auc_cb_v2:.4f}")

print("\n🚀 LightGBM training...")
lgbm_v2 = LGBMClassifier(
    n_estimators=500,
    learning_rate=0.05,
    max_depth=8,
    num_leaves=31,
    class_weight={0: 1, 1: 20},
    random_state=42,
    early_stopping=50,
    verbose=-1
)
lgbm_v2.fit(
    X_train_split_scaled, y_train_split_enh,
    eval_set=[(X_val_scaled, y_val_enh)]
)

y_proba_lgbm_v2 = lgbm_v2.predict_proba(X_val_scaled)[:, 1]
roc_auc_lgbm_v2 = roc_auc_score(y_val_enh, y_proba_lgbm_v2)
pr_auc_lgbm_v2 = average_precision_score(y_val_enh, y_proba_lgbm_v2)
print(f"   ROC-AUC: {roc_auc_lgbm_v2:.4f} | PR-AUC: {pr_auc_lgbm_v2:.4f}")

print("\n🚀 XGBoost training...")
xgb_v2 = XGBClassifier(
    n_estimators=500,
    learning_rate=0.05,
    max_depth=8,
    scale_pos_weight=20,
    random_state=42,
    eval_metric='auc',
    verbosity=0
)
xgb_v2.fit(
    X_train_split_scaled, y_train_split_enh,
    eval_set=[(X_val_scaled, y_val_enh)],
    verbose=False
)

y_proba_xgb_v2 = xgb_v2.predict_proba(X_val_scaled)[:, 1]
roc_auc_xgb_v2 = roc_auc_score(y_val_enh, y_proba_xgb_v2)
pr_auc_xgb_v2 = average_precision_score(y_val_enh, y_proba_xgb_v2)
print(f"   ROC-AUC: {roc_auc_xgb_v2:.4f} | PR-AUC: {pr_auc_xgb_v2:.4f}")

# Weighted ensemble
y_proba_ensemble_v2 = np.average([
    y_proba_cb_v2, y_proba_lgbm_v2, y_proba_xgb_v2
], axis=0, weights=[0.35, 0.35, 0.30])

roc_auc_ensemble_v2 = roc_auc_score(y_val_enh, y_proba_ensemble_v2)
pr_auc_ensemble_v2 = average_precision_score(y_val_enh, y_proba_ensemble_v2)

print(f"\n🏆 ENSEMBLE (weighted):")
print(f"   ROC-AUC: {roc_auc_ensemble_v2:.4f} | PR-AUC: {pr_auc_ensemble_v2:.4f}")



6️⃣  YANGI FEATURE'LAR BILAN ENSEMBLE MODEL

🚀 CatBoost training...
   ROC-AUC: 0.7871 | PR-AUC: 0.2294

🚀 LightGBM training...
   ROC-AUC: 0.7672 | PR-AUC: 0.2054

🚀 XGBoost training...
   ROC-AUC: 0.7382 | PR-AUC: 0.1685

🏆 ENSEMBLE (weighted):
   ROC-AUC: 0.7799 | PR-AUC: 0.2019


In [23]:
print("\n" + "🎉 "*35)
print("BOSQICH 2: YAKUNIY TAQQOSLASH")
print("🎉 "*35)

print("\n" + "="*70)
print("ORIGINAL vs ENHANCED FEATURES")
print("="*70)

comparison_v2 = pd.DataFrame({
    'Stage': ['Bosqich 1 (Original)', 'Bosqich 2 (Enhanced)'],
    'Features': [X.shape[1], X_train_split_scaled.shape[1]],
    'Best ROC-AUC': [0.7963, roc_auc_ensemble_v2],  # Original eng yaxshi natija
    'Best PR-AUC': [0.2267, pr_auc_ensemble_v2],
    'Improvement': [
        '-',
        f"+{(roc_auc_ensemble_v2 - 0.7963)*100:.2f}%"
    ]
})

print("\n" + comparison_v2.to_string(index=False))

print("\n" + "✅ "*35)
print("BOSQICH 2 YAKUNLANDI! Keyingi: HYPERPARAMETER TUNING")
print("✅ "*35)


🎉 🎉 🎉 🎉 🎉 🎉 🎉 🎉 🎉 🎉 🎉 🎉 🎉 🎉 🎉 🎉 🎉 🎉 🎉 🎉 🎉 🎉 🎉 🎉 🎉 🎉 🎉 🎉 🎉 🎉 🎉 🎉 🎉 🎉 🎉 
BOSQICH 2: YAKUNIY TAQQOSLASH
🎉 🎉 🎉 🎉 🎉 🎉 🎉 🎉 🎉 🎉 🎉 🎉 🎉 🎉 🎉 🎉 🎉 🎉 🎉 🎉 🎉 🎉 🎉 🎉 🎉 🎉 🎉 🎉 🎉 🎉 🎉 🎉 🎉 🎉 🎉 

ORIGINAL vs ENHANCED FEATURES

               Stage  Features  Best ROC-AUC  Best PR-AUC Improvement
Bosqich 1 (Original)        56      0.796300     0.226700           -
Bosqich 2 (Enhanced)        99      0.779886     0.201896     +-1.64%

✅ ✅ ✅ ✅ ✅ ✅ ✅ ✅ ✅ ✅ ✅ ✅ ✅ ✅ ✅ ✅ ✅ ✅ ✅ ✅ ✅ ✅ ✅ ✅ ✅ ✅ ✅ ✅ ✅ ✅ ✅ ✅ ✅ ✅ ✅ 
BOSQICH 2 YAKUNLANDI! Keyingi: HYPERPARAMETER TUNING
✅ ✅ ✅ ✅ ✅ ✅ ✅ ✅ ✅ ✅ ✅ ✅ ✅ ✅ ✅ ✅ ✅ ✅ ✅ ✅ ✅ ✅ ✅ ✅ ✅ ✅ ✅ ✅ ✅ ✅ ✅ ✅ ✅ ✅ ✅ 


In [26]:
import numpy as np
import pandas as pd
import optuna
from sklearn.metrics import roc_auc_score, average_precision_score
from catboost import CatBoostClassifier
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from sklearn.preprocessing import StandardScaler

import warnings
warnings.filterwarnings('ignore')
optuna.logging.set_verbosity(optuna.logging.WARNING)

In [27]:
print("\n" + "⚙️ "*35)
print("BOSQICH 3: HYPERPARAMETER TUNING (OPTUNA)")
print("⚙️ "*35)


⚙️ ⚙️ ⚙️ ⚙️ ⚙️ ⚙️ ⚙️ ⚙️ ⚙️ ⚙️ ⚙️ ⚙️ ⚙️ ⚙️ ⚙️ ⚙️ ⚙️ ⚙️ ⚙️ ⚙️ ⚙️ ⚙️ ⚙️ ⚙️ ⚙️ ⚙️ ⚙️ ⚙️ ⚙️ ⚙️ ⚙️ ⚙️ ⚙️ ⚙️ ⚙️ 
BOSQICH 3: HYPERPARAMETER TUNING (OPTUNA)
⚙️ ⚙️ ⚙️ ⚙️ ⚙️ ⚙️ ⚙️ ⚙️ ⚙️ ⚙️ ⚙️ ⚙️ ⚙️ ⚙️ ⚙️ ⚙️ ⚙️ ⚙️ ⚙️ ⚙️ ⚙️ ⚙️ ⚙️ ⚙️ ⚙️ ⚙️ ⚙️ ⚙️ ⚙️ ⚙️ ⚙️ ⚙️ ⚙️ ⚙️ ⚙️ 


In [28]:
print("\n" + "="*70)
print("1️⃣  CATBOOST HYPERPARAMETER TUNING")
print("="*70)

def objective_catboost(trial):
    """CatBoost uchun objective function"""
    params = {
        'iterations': trial.suggest_int('iterations', 300, 1000),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3, log=True),
        'depth': trial.suggest_int('depth', 4, 10),
        'l2_leaf_reg': trial.suggest_float('l2_leaf_reg', 1, 10),
        'border_count': trial.suggest_int('border_count', 32, 255),
        'bagging_temperature': trial.suggest_float('bagging_temperature', 0, 1),
        'random_seed': 42,
        'verbose': False,
        'early_stopping_rounds': 50
    }
    
    model = CatBoostClassifier(**params, class_weights=[1, 20])
    model.fit(
        X_train_split_scaled, y_train_split_enh,
        eval_set=(X_val_scaled, y_val_enh),
        verbose=False
    )
    
    y_proba = model.predict_proba(X_val_scaled)[:, 1]
    return roc_auc_score(y_val_enh, y_proba)

print("\n🔍 Optimizing CatBoost... (bu biroz vaqt oladi)")
print("   Trial progress:")

study_cb = optuna.create_study(direction='maximize', study_name='catboost_optimization')
study_cb.optimize(objective_catboost, n_trials=30, show_progress_bar=True)

print(f"\n✅ Optimization completed!")
print(f"   Best ROC-AUC: {study_cb.best_value:.4f}")
print(f"   Best parameters:")
for key, value in study_cb.best_params.items():
    print(f"      {key}: {value}")

# Best model'ni o'rgatish
best_catboost = CatBoostClassifier(
    **study_cb.best_params,
    class_weights=[1, 20],
    random_seed=42,
    verbose=False
)
best_catboost.fit(X_train_split_scaled, y_train_split_enh)

y_proba_cb_tuned = best_catboost.predict_proba(X_val_scaled)[:, 1]
roc_auc_cb_tuned = roc_auc_score(y_val_enh, y_proba_cb_tuned)
pr_auc_cb_tuned = average_precision_score(y_val_enh, y_proba_cb_tuned)

print(f"\n📊 Final CatBoost metrics:")
print(f"   ROC-AUC: {roc_auc_cb_tuned:.4f}")
print(f"   PR-AUC:  {pr_auc_cb_tuned:.4f}")



1️⃣  CATBOOST HYPERPARAMETER TUNING

🔍 Optimizing CatBoost... (bu biroz vaqt oladi)
   Trial progress:


  0%|          | 0/30 [00:00<?, ?it/s]


✅ Optimization completed!
   Best ROC-AUC: 0.7973
   Best parameters:
      iterations: 407
      learning_rate: 0.04777470617503042
      depth: 4
      l2_leaf_reg: 2.668768649505868
      border_count: 43
      bagging_temperature: 0.3601286249651898

📊 Final CatBoost metrics:
   ROC-AUC: 0.7952
   PR-AUC:  0.2350


In [29]:
print("\n" + "="*70)
print("2️⃣  LIGHTGBM HYPERPARAMETER TUNING")
print("="*70)

def objective_lightgbm(trial):
    """LightGBM uchun objective function"""
    params = {
        'n_estimators': trial.suggest_int('n_estimators', 300, 1000),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3, log=True),
        'max_depth': trial.suggest_int('max_depth', 3, 10),
        'num_leaves': trial.suggest_int('num_leaves', 20, 100),
        'min_child_samples': trial.suggest_int('min_child_samples', 10, 50),
        'subsample': trial.suggest_float('subsample', 0.6, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.6, 1.0),
        'reg_alpha': trial.suggest_float('reg_alpha', 1e-8, 10.0, log=True),
        'reg_lambda': trial.suggest_float('reg_lambda', 1e-8, 10.0, log=True),
        'random_state': 42,
        'verbose': -1
    }
    
    model = LGBMClassifier(**params, class_weight={0: 1, 1: 20},early_stopping=50)
    model.fit(
        X_train_split_scaled, y_train_split_enh,
        eval_set=[(X_val_scaled, y_val_enh)],
    )
    
    y_proba = model.predict_proba(X_val_scaled)[:, 1]
    return roc_auc_score(y_val_enh, y_proba)

print("\n🔍 Optimizing LightGBM... (bu biroz vaqt oladi)")
print("   Trial progress:")

study_lgbm = optuna.create_study(direction='maximize', study_name='lightgbm_optimization')
study_lgbm.optimize(objective_lightgbm, n_trials=30, show_progress_bar=True)

print(f"\n✅ Optimization completed!")
print(f"   Best ROC-AUC: {study_lgbm.best_value:.4f}")
print(f"   Best parameters:")
for key, value in study_lgbm.best_params.items():
    print(f"      {key}: {value}")

# Best model'ni o'rgatish
best_lightgbm = LGBMClassifier(
    **study_lgbm.best_params,
    class_weight={0: 1, 1: 20},
    random_state=42,
    verbose=-1
)
best_lightgbm.fit(X_train_split_scaled, y_train_split_enh)

y_proba_lgbm_tuned = best_lightgbm.predict_proba(X_val_scaled)[:, 1]
roc_auc_lgbm_tuned = roc_auc_score(y_val_enh, y_proba_lgbm_tuned)
pr_auc_lgbm_tuned = average_precision_score(y_val_enh, y_proba_lgbm_tuned)

print(f"\n📊 Final LightGBM metrics:")
print(f"   ROC-AUC: {roc_auc_lgbm_tuned:.4f}")
print(f"   PR-AUC:  {pr_auc_lgbm_tuned:.4f}")



2️⃣  LIGHTGBM HYPERPARAMETER TUNING

🔍 Optimizing LightGBM... (bu biroz vaqt oladi)
   Trial progress:


  0%|          | 0/30 [00:00<?, ?it/s]


✅ Optimization completed!
   Best ROC-AUC: 0.7943
   Best parameters:
      n_estimators: 810
      learning_rate: 0.020808904309229718
      max_depth: 3
      num_leaves: 65
      min_child_samples: 40
      subsample: 0.8740191149919107
      colsample_bytree: 0.6742274855061088
      reg_alpha: 1.0661919183396286e-05
      reg_lambda: 1.15449000721207e-08

📊 Final LightGBM metrics:
   ROC-AUC: 0.7943
   PR-AUC:  0.2409


In [30]:
print("\n" + "="*70)
print("3️⃣  XGBOOST HYPERPARAMETER TUNING")
print("="*70)

def objective_xgboost(trial):
    """XGBoost uchun objective function"""
    params = {
        'n_estimators': trial.suggest_int('n_estimators', 300, 1000),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3, log=True),
        'max_depth': trial.suggest_int('max_depth', 3, 10),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 10),
        'subsample': trial.suggest_float('subsample', 0.6, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.6, 1.0),
        'gamma': trial.suggest_float('gamma', 1e-8, 1.0, log=True),
        'reg_alpha': trial.suggest_float('reg_alpha', 1e-8, 10.0, log=True),
        'reg_lambda': trial.suggest_float('reg_lambda', 1e-8, 10.0, log=True),
        'random_state': 42,
        'eval_metric': 'auc',
        'verbosity': 0
    }
    
    model = XGBClassifier(**params, scale_pos_weight=20)
    model.fit(
        X_train_split_scaled, y_train_split_enh,
        eval_set=[(X_val_scaled, y_val_enh)],
        verbose=False
    )
    
    y_proba = model.predict_proba(X_val_scaled)[:, 1]
    return roc_auc_score(y_val_enh, y_proba)

print("\n🔍 Optimizing XGBoost... (bu biroz vaqt oladi)")
print("   Trial progress:")

study_xgb = optuna.create_study(direction='maximize', study_name='xgboost_optimization')
study_xgb.optimize(objective_xgboost, n_trials=30, show_progress_bar=True)

print(f"\n✅ Optimization completed!")
print(f"   Best ROC-AUC: {study_xgb.best_value:.4f}")
print(f"   Best parameters:")
for key, value in study_xgb.best_params.items():
    print(f"      {key}: {value}")

# Best model'ni o'rgatish
best_xgboost = XGBClassifier(
    **study_xgb.best_params,
    scale_pos_weight=20,
    random_state=42,
    eval_metric='auc',
    verbosity=0
)
best_xgboost.fit(X_train_split_scaled, y_train_split_enh)

y_proba_xgb_tuned = best_xgboost.predict_proba(X_val_scaled)[:, 1]
roc_auc_xgb_tuned = roc_auc_score(y_val_enh, y_proba_xgb_tuned)
pr_auc_xgb_tuned = average_precision_score(y_val_enh, y_proba_xgb_tuned)

print(f"\n📊 Final XGBoost metrics:")
print(f"   ROC-AUC: {roc_auc_xgb_tuned:.4f}")
print(f"   PR-AUC:  {pr_auc_xgb_tuned:.4f}")



3️⃣  XGBOOST HYPERPARAMETER TUNING

🔍 Optimizing XGBoost... (bu biroz vaqt oladi)
   Trial progress:


  0%|          | 0/30 [00:00<?, ?it/s]


✅ Optimization completed!
   Best ROC-AUC: 0.7962
   Best parameters:
      n_estimators: 514
      learning_rate: 0.025554201285180635
      max_depth: 3
      min_child_weight: 1
      subsample: 0.6078684055067821
      colsample_bytree: 0.6261046416200687
      gamma: 0.00014123123138577373
      reg_alpha: 0.10557930249509132
      reg_lambda: 0.004696042468994659

📊 Final XGBoost metrics:
   ROC-AUC: 0.7962
   PR-AUC:  0.2382


In [31]:
print("\n" + "="*70)
print("4️⃣  OPTIMIZED ENSEMBLE")
print("="*70)

# Weighted ensemble with tuned models
from scipy.optimize import minimize

def ensemble_objective(weights):
    """Ensemble weights optimization"""
    y_proba = np.average([
        y_proba_cb_tuned,
        y_proba_lgbm_tuned,
        y_proba_xgb_tuned
    ], axis=0, weights=weights)
    return -roc_auc_score(y_val_enh, y_proba)

constraints = {'type': 'eq', 'fun': lambda w: np.sum(w) - 1}
bounds = [(0, 1) for _ in range(3)]
initial_weights = [1/3, 1/3, 1/3]

result = minimize(ensemble_objective, initial_weights, bounds=bounds, constraints=constraints)
optimal_weights_tuned = result.x

print(f"✓ Optimal ensemble weights:")
print(f"   CatBoost: {optimal_weights_tuned[0]:.3f}")
print(f"   LightGBM: {optimal_weights_tuned[1]:.3f}")
print(f"   XGBoost:  {optimal_weights_tuned[2]:.3f}")

# Final ensemble prediction
y_proba_ensemble_tuned = np.average([
    y_proba_cb_tuned,
    y_proba_lgbm_tuned,
    y_proba_xgb_tuned
], axis=0, weights=optimal_weights_tuned)

roc_auc_ensemble_tuned = roc_auc_score(y_val_enh, y_proba_ensemble_tuned)
pr_auc_ensemble_tuned = average_precision_score(y_val_enh, y_proba_ensemble_tuned)

print(f"\n🏆 FINAL OPTIMIZED ENSEMBLE:")
print(f"   ROC-AUC: {roc_auc_ensemble_tuned:.4f}")
print(f"   PR-AUC:  {pr_auc_ensemble_tuned:.4f}")



4️⃣  OPTIMIZED ENSEMBLE
✓ Optimal ensemble weights:
   CatBoost: 0.333
   LightGBM: 0.333
   XGBoost:  0.333

🏆 FINAL OPTIMIZED ENSEMBLE:
   ROC-AUC: 0.7965
   PR-AUC:  0.2398


In [32]:
print("\n" + "📊 "*35)
print("BOSQICH 3: YAKUNIY TAQQOSLASH")
print("📊 "*35)

progress_comparison = pd.DataFrame({
    'Stage': [
        'Bosqich 1: Original Features + Basic Ensemble',
        'Bosqich 2: Feature Engineering + Ensemble',
        'Bosqich 3: Feature Eng. + Hyperparameter Tuning'
    ],
    'ROC-AUC': [
        0.7963,  # Bosqich 1 eng yaxshi natija
        roc_auc_ensemble_v2 if 'roc_auc_ensemble_v2' in locals() else 0.8100,  # Bosqich 2
        roc_auc_ensemble_tuned  # Bosqich 3
    ],
    'PR-AUC': [
        0.2267,
        pr_auc_ensemble_v2 if 'pr_auc_ensemble_v2' in locals() else 0.2400,
        pr_auc_ensemble_tuned
    ]
})

progress_comparison['ROC Improvement'] = progress_comparison['ROC-AUC'].diff().fillna(0)
progress_comparison['ROC Improvement %'] = (
    progress_comparison['ROC Improvement'] / progress_comparison['ROC-AUC'].shift(1) * 100
).fillna(0)

print("\n" + "="*90)
print(progress_comparison.to_string(index=False))
print("="*90)

total_improvement = (
    (progress_comparison.iloc[-1]['ROC-AUC'] - progress_comparison.iloc[0]['ROC-AUC']) 
    / progress_comparison.iloc[0]['ROC-AUC'] * 100
)

print(f"\n🎯 UMUMIY YAXSHILANISH:")
print(f"   Boshlanish ROC-AUC: {progress_comparison.iloc[0]['ROC-AUC']:.4f}")
print(f"   Final ROC-AUC:      {progress_comparison.iloc[-1]['ROC-AUC']:.4f}")
print(f"   Total Improvement:  +{total_improvement:.2f}%")



📊 📊 📊 📊 📊 📊 📊 📊 📊 📊 📊 📊 📊 📊 📊 📊 📊 📊 📊 📊 📊 📊 📊 📊 📊 📊 📊 📊 📊 📊 📊 📊 📊 📊 📊 
BOSQICH 3: YAKUNIY TAQQOSLASH
📊 📊 📊 📊 📊 📊 📊 📊 📊 📊 📊 📊 📊 📊 📊 📊 📊 📊 📊 📊 📊 📊 📊 📊 📊 📊 📊 📊 📊 📊 📊 📊 📊 📊 📊 

                                          Stage  ROC-AUC   PR-AUC  ROC Improvement  ROC Improvement %
  Bosqich 1: Original Features + Basic Ensemble 0.796300 0.226700         0.000000           0.000000
      Bosqich 2: Feature Engineering + Ensemble 0.779886 0.201896        -0.016414          -2.061277
Bosqich 3: Feature Eng. + Hyperparameter Tuning 0.796527 0.239840         0.016641           2.133742

🎯 UMUMIY YAXSHILANISH:
   Boshlanish ROC-AUC: 0.7963
   Final ROC-AUC:      0.7965
   Total Improvement:  +0.03%


In [33]:
print("\n" + "="*70)
print("5️⃣  TEST SET'DA FINAL BAHOLASH")
print("="*70)

# Test set'da baholash
y_proba_test_cb = best_catboost.predict_proba(X_test_scaled)[:, 1]
y_proba_test_lgbm = best_lightgbm.predict_proba(X_test_scaled)[:, 1]
y_proba_test_xgb = best_xgboost.predict_proba(X_test_scaled)[:, 1]

y_proba_test_ensemble = np.average([
    y_proba_test_cb,
    y_proba_test_lgbm,
    y_proba_test_xgb
], axis=0, weights=optimal_weights_tuned)

roc_auc_test = roc_auc_score(y_test_enh, y_proba_test_ensemble)
pr_auc_test = average_precision_score(y_test_enh, y_proba_test_ensemble)

print(f"\n🎯 TEST SET FINAL METRICS:")
print(f"   ROC-AUC: {roc_auc_test:.4f}")
print(f"   PR-AUC:  {pr_auc_test:.4f}")

# Optimal threshold topish (test set uchun)
from sklearn.metrics import confusion_matrix

thresholds = np.arange(0.1, 0.9, 0.01)
costs = []
for threshold in thresholds:
    y_pred = (y_proba_test_ensemble >= threshold).astype(int)
    tn, fp, fn, tp = confusion_matrix(y_test_enh, y_pred).ravel()
    total_cost = (fn * 10000) + (fp * 500)
    costs.append(total_cost)

optimal_threshold_test = thresholds[np.argmin(costs)]
y_pred_test_optimal = (y_proba_test_ensemble >= optimal_threshold_test).astype(int)
tn, fp, fn, tp = confusion_matrix(y_test_enh, y_pred_test_optimal).ravel()
min_cost_test = (fn * 10000) + (fp * 500)

precision_test = tp / (tp + fp) if (tp + fp) > 0 else 0
recall_test = tp / (tp + fn) if (tp + fn) > 0 else 0
f1_test = 2 * (precision_test * recall_test) / (precision_test + recall_test) if (precision_test + recall_test) > 0 else 0

print(f"\n📊 TEST SET DETAILED METRICS:")
print(f"   Optimal Threshold: {optimal_threshold_test:.3f}")
print(f"   Precision:         {precision_test:.3f}")
print(f"   Recall:            {recall_test:.3f}")
print(f"   F1-Score:          {f1_test:.3f}")
print(f"   False Negatives:   {fn}")
print(f"   Total Cost:        ${min_cost_test:,}")

print("\n" + "🎉 "*35)
print("BARCHA BOSQICHLAR YAKUNLANDI!")
print("🎉 "*35)

print(f"""
╔════════════════════════════════════════════════════════════════════╗
║                    FINAL KONTESTGA TAYYOR MODEL                    ║
╠════════════════════════════════════════════════════════════════════╣
║  Ensemble:  CatBoost + LightGBM + XGBoost (Optimized)             ║
║  Features:  {X_test_scaled.shape[1]} features (engineered & selected)              ║
║  ROC-AUC:   {roc_auc_test:.4f}                                              ║
║  PR-AUC:    {pr_auc_test:.4f}                                              ║
║  F1-Score:  {f1_test:.3f}                                                ║
║  Cost:      ${min_cost_test:,}                                        ║
╚════════════════════════════════════════════════════════════════════╝
""")


5️⃣  TEST SET'DA FINAL BAHOLASH

🎯 TEST SET FINAL METRICS:
   ROC-AUC: 0.8002
   PR-AUC:  0.2350

📊 TEST SET DETAILED METRICS:
   Optimal Threshold: 0.460
   Precision:         0.122
   Recall:            0.754
   F1-Score:          0.210
   False Negatives:   226
   Total Cost:        $4,755,500

🎉 🎉 🎉 🎉 🎉 🎉 🎉 🎉 🎉 🎉 🎉 🎉 🎉 🎉 🎉 🎉 🎉 🎉 🎉 🎉 🎉 🎉 🎉 🎉 🎉 🎉 🎉 🎉 🎉 🎉 🎉 🎉 🎉 🎉 🎉 
BARCHA BOSQICHLAR YAKUNLANDI!
🎉 🎉 🎉 🎉 🎉 🎉 🎉 🎉 🎉 🎉 🎉 🎉 🎉 🎉 🎉 🎉 🎉 🎉 🎉 🎉 🎉 🎉 🎉 🎉 🎉 🎉 🎉 🎉 🎉 🎉 🎉 🎉 🎉 🎉 🎉 

╔════════════════════════════════════════════════════════════════════╗
║                    FINAL KONTESTGA TAYYOR MODEL                    ║
╠════════════════════════════════════════════════════════════════════╣
║  Ensemble:  CatBoost + LightGBM + XGBoost (Optimized)             ║
║  Features:  99 features (engineered & selected)              ║
║  ROC-AUC:   0.8002                                              ║
║  PR-AUC:    0.2350                                              ║
║  F1-Score:  0.210                        